# Variational Bayes with Pseudodata Prior

In [1]:
using Statistics, Distributions
using SpecialFunctions, LinearAlgebra
import Base.Iterators: product
import Clustering: kmeans

In [2]:
include("./src/Misc.jl")
using .Misc

In [3]:
include("./src/BayesianCausality.jl")
import .BayesianCausality

## Trying the new library functions

In [4]:
R, M = 3, 2
Tᵖ = Int((M+1)*M/2)
λ₁ᵣ, λ₂ᵣ, γᵣ = 1., 1., 2.
m₁ᵣ, m₂ᵣ = 0., 0.
a₁ᵣ, a₂ᵣ = 1., 1.
b₁ᵣ, b₂ᵣ = 1., 1.;

In [5]:
X = (rand(20, 2) .- 0.5).*5;
pseudodata_type = "random";

In [8]:
BayesianCausality.vb_causal_likelihoods(X[1:end,1],X[1:end,2],R; M=1, EPOCHS=200, parameters_pair=Dict(), 
    norm=true, pseudodata_type="random", m_zero=true)

String: "M has to be larger than 1"

In [8]:
X_p = BayesianCausality.get_pseudodata(X, R, M; pseudodata_type=pseudodata_type)

UndefVarError: UndefVarError: pseudata_type not defined

In [11]:
X_p = BayesianCausality.get_pseudodata(X, R, M; pseudodata_type=pseudodata_type)
X_p[:,:,2] .= 1.
BayesianCausality.make_param(X_p, 1)

3
1


Dict{Symbol,Array{Float64,N} where N} with 9 entries:
  :γ  => [3.0, 3.0, 3.0]
  :Λ₁ => [3.0, 3.0, 3.0]
  :Λ₂ => [3.0; 3.0; 3.0]
  :b₁ => [0.0, 0.0, 0.0]
  :a₁ => [1.5, 1.5, 1.5]
  :a₂ => [1.5, 1.5, 1.5]
  :m₂ => [1.0; 1.0; 1.0]
  :b₂ => [0.0, 0.0, 0.0]
  :m₁ => [0.0, 0.0, 0.0]

## Functions

In [16]:
mutable struct Particle{ℜ <: Real}
    r::Array{Int,1}
    Σ_R::Array{ℜ,1}
    Σ_X₁::Array{ℜ,1}
    Σ_X₁²::Array{ℜ,1}
    Σ_X₂²::Array{ℜ,1}
    Σ_X₂ϕ::Array{ℜ,2}
    Σ_ϕϕᵀ::Array{ℜ,3}
    log_πₜ::ℜ
    
    function Particle(X::Array{ℜ,2}, R::Int; M::Int=1) where {ℜ <: Real}
        T, N = size(X)
        return new{ℜ}(zeros(Int,T), zeros(ℜ,R), zeros(ℜ,R), zeros(ℜ,R),
                      zeros(ℜ,R), zeros(ℜ,R,M), zeros(ℜ,R,M,M), 0.0)
    end
    
    function Particle(r::Array{Int,1}, Σ_R::Array{Int,1}, Σ_X₁::Array{ℜ,1}, Σ_X₁²::Array{ℜ,1},
                      Σ_X₂²::Array{ℜ,1}, Σ_X₂ϕ::Array{ℜ,2}, Σ_ϕϕᵀ::Array{ℜ,3}, log_πₜ::ℜ) where {ℜ <: Real}
        return new{ℜ}(copy(r), copy(Σ_R), copy(Σ_X₁), copy(Σ_X₁²),
                      copy(Σ_X₂²), copy(Σ_X₂ϕ), copy(Σ_ϕϕᵀ), log_πₜ)
    end
    
    function Particle(p::Particle{ℜ}) where {ℜ <: Real}
        return new{ℜ}(copy(p.r), copy(p.Σ_R), copy(p.Σ_X₁), copy(p.Σ_X₁²),
                      copy(p.Σ_X₂²), copy(p.Σ_X₂ϕ), copy(p.Σ_ϕϕᵀ), p.log_πₜ)
    end 
end

function conduct_vb(X::Array{ℜ,2}, R::Int, M::Int, Xᵖ::Array{ℜ,3}; EPOCHS::Int) where {ℜ <: Real}
    params_new = make_param(Xᵖ::Array{ℜ,3},M::Int)
    return v_bayes_pseudodata(X, R; M=M, EPOCHS=EPOCHS, params_new...)
end

function conduct_vb(X::Array{ℜ,2}, R::Int, M::Int; γᵣ, m₁ᵣ, m₂ᵣ, 
        λ₁ᵣ, λ₂ᵣ, a₁ᵣ, a₂ᵣ, b₁ᵣ, b₂ᵣ, EPOCHS) where {ℜ <: Real}
    params_original = make_param(R, M; γᵣ=γᵣ, m₁ᵣ=m₁ᵣ, m₂ᵣ=m₂ᵣ, λ₁ᵣ=λ₁ᵣ, λ₂ᵣ=λ₂ᵣ,
        a₁ᵣ=a₁ᵣ, a₂ᵣ=a₂ᵣ, b₁ᵣ=b₁ᵣ, b₂ᵣ=b₂ᵣ)
    return BayesianCausality.v_bayes(X, R; M=M, EPOCHS=EPOCHS, params_original...)
end

function make_param(Xᵖ::Array{ℜ,3},M::Int) where {ℜ <: Real}
    R, T = size(Xᵖ)[1:end-1]
    #if T != Int((M+1)*M/2)
    #    throw("Prior data size not correct")
    #end
    X = reshape(Xᵖ,:,2)
    p = Particle(X, R; M=M)
    ϕ = zeros(M)
    
    for r ∈ 1:R, t ∈ 1:T
        x₁, x₂ = Xᵖ[r,t,:]
        ϕ .= x₁.^(0:M-1)
        p.Σ_R[r] += 1.0
        p.Σ_X₁[r] += x₁
        p.Σ_X₁²[r] += x₁*x₁
        p.Σ_X₂²[r] += x₂*x₂
        p.Σ_X₂ϕ[r,:] .+= x₂*ϕ
        p.Σ_ϕϕᵀ[r,:,:] .+= ϕ*ϕ'
    end
    
    γ = zeros(R)
    Λ₁, m₁ = zeros(R), zeros(R)
    a₁, b₁ = zeros(R), zeros(R)
    Λ₂, m₂ = zeros(R,M,M), zeros(R,M) 
    a₂, b₂ = zeros(R), zeros(R)
    
    for r ∈ 1:R
        γ[r] = p.Σ_R[r]
        Λ₁[r] = p.Σ_R[r]
        m₁[r] = p.Σ_X₁[r] / Λ₁[r]
        a₁[r] = p.Σ_R[r]/2
        b₁[r] = (p.Σ_X₁²[r] - Λ₁[r]*m₁[r]*m₁[r])/2
        
        Λ₂[r,:,:] .= p.Σ_ϕϕᵀ[r,:,:]
        m₂[r,:] .= Λ₂[r,:,:] \ p.Σ_X₂ϕ[r,:]
        a₂[r] = p.Σ_R[r]/2
        b₂[r] = (p.Σ_X₂²[r] - m₂[r,:]'*Λ₂[r,:,:]*m₂[r,:])/2
    end
    
    param = Dict(:γ=>γ, :m₁=>m₁, :m₂=>m₂, :Λ₁=>Λ₁, 
    :Λ₂=>Λ₂, :a₁=>a₁, :a₂=>a₂, :b₁=>b₁, :b₂=>b₂);
    
    return param
end

function make_param(R::Int, M::Int; γᵣ, m₁ᵣ, m₂ᵣ, λ₁ᵣ, λ₂ᵣ, a₁ᵣ, a₂ᵣ, b₁ᵣ, b₂ᵣ)
    return  Dict(:γ=>fill(γᵣ,R), :m₁=>zeros(R), :m₂=>zeros(R,M), :Λ₁=>fill(λ₁ᵣ,R), 
        :Λ₂=>zeros(R,M,M) .+ λ₂ᵣ*reshape(diagm(0 => ones(M)),1,M,M), :a₁=>fill(a₁ᵣ,R), 
        :a₂=>fill(a₂ᵣ,R), :b₁=>fill(b₁ᵣ,R), :b₂=>fill(b₂ᵣ,R));
end

function v_bayes_pseudodata(X::Array{ℜ,2}, R::Ƶ=1; M::Ƶ=1, γ::Array{ℜ,1}, m₁::Array{ℜ,1}, m₂::Array{ℜ,2},
        Λ₁::Array{ℜ,1}, Λ₂::Array{ℜ,3}, a₁::Array{ℜ,1}, a₂::Array{ℜ,1}, 
        b₁::Array{ℜ,1}, b₂::Array{ℜ,1}, EPOCHS=1) where {ℜ<:Real, Ƶ<:Integer}
    T, N = size(X)
    ELBO = zeros(EPOCHS)
    
    p = Particle(X, R; M=M)
    p.r .=  T < R ? [1:T...] : R > 1 ? kmeans(Array(X'),R).assignments : ones(Int,T)
    X₁, X₂ = X[:,1], X[:,2]
    
    γʰ = zeros(R)
    Λ₁ʰ, m₁ʰ = zeros(R), zeros(R)
    a₁ʰ, b₁ʰ = zeros(R), zeros(R)
    Λ₂ʰ, m₂ʰ = zeros(R,M,M), zeros(R,M) 
    a₂ʰ, b₂ʰ = zeros(R), zeros(R)
    
    ϕ, log_q, q = zeros(M), zeros(R), zeros(R)
    ρ₁, log_ρ₁, ρ₂, log_ρ₂, log_θ = zeros(R), zeros(R), zeros(R), zeros(R), zeros(R)
    
    for (r, x₁, x₂) ∈ zip(p.r, X₁, X₂)
        ϕ .= x₁.^(0:M-1)
        p.Σ_R[r] += 1.0
        p.Σ_X₁[r] += x₁
        p.Σ_X₁²[r] += x₁*x₁
        p.Σ_X₂²[r] += x₂*x₂
        p.Σ_X₂ϕ[r,:] .+= x₂*ϕ
        p.Σ_ϕϕᵀ[r,:,:] .+= ϕ*ϕ'
    end
    
    for eph ∈ 1:EPOCHS
        for r ∈ 1:R
            γʰ[r] = γ[r] + p.Σ_R[r]
            Λ₁ʰ[r] = Λ₁[r] + p.Σ_R[r]
            m₁ʰ[r] = (Λ₁[r]*m₁[r] + p.Σ_X₁[r]) / Λ₁ʰ[r]
            a₁ʰ[r] = a₁[r] + p.Σ_R[r]/2
            b₁ʰ[r] = b₁[r] + (p.Σ_X₁²[r] - Λ₁ʰ[r]*m₁ʰ[r]*m₁ʰ[r] + Λ₁[r]*m₁[r]*m₁[r])/2
        
            Λ₂ʰ[r,:,:] .= Λ₂[r,:,:] .+ p.Σ_ϕϕᵀ[r,:,:]
            m₂ʰ[r,:] .= Λ₂ʰ[r,:,:] \ (Λ₂[r,:,:]*m₂[r,:] .+ p.Σ_X₂ϕ[r,:])
            a₂ʰ[r] = a₂[r] + p.Σ_R[r]/2
            b₂ʰ[r] = b₂[r] + (p.Σ_X₂²[r] - m₂ʰ[r,:]'*Λ₂ʰ[r,:,:]*m₂ʰ[r,:] + m₂[r,:]'*Λ₂[r,:,:]*m₂[r,:])/2
            
            log_ρ₁[r] = digamma(a₁ʰ[r]) - log(b₁ʰ[r])
            ρ₁[r] = a₁ʰ[r]/b₁ʰ[r]
            log_ρ₂[r] = digamma(a₂ʰ[r]) - log(b₂ʰ[r])
            ρ₂[r] = a₂ʰ[r]/b₂ʰ[r]
            log_θ[r] = digamma(γʰ[r]) - digamma(sum(γ)+T)
            
            ELBO[eph] += (γ[r]-γʰ[r])*log_θ[r]
            
            ELBO[eph] += a₁[r]*log(b₁[r]) - a₁ʰ[r]*log(b₁ʰ[r])
            ELBO[eph] += lgamma(a₁ʰ[r]) - lgamma(a₁[r])
            ELBO[eph] += (log(Λ₁[r]) - log(Λ₁ʰ[r]))/2
            ELBO[eph] += (1 - Λ₁[r]/Λ₁ʰ[r])/2
            ELBO[eph] += (a₁[r] - a₁ʰ[r])*log_ρ₁[r] - (b₁[r] - b₁ʰ[r])*ρ₁[r]
            ELBO[eph] -= (ρ₁[r]*Λ₁[r]*(m₁ʰ[r] - m₁[r])^2)/2
                
            ELBO[eph] += a₂[r]*log(b₂[r]) - a₂ʰ[r]*log(b₂ʰ[r])
            ELBO[eph] += lgamma(a₂ʰ[r]) - lgamma(a₂[r])
            ELBO[eph] += (logdet(Λ₂[r,:,:]) - logdet(Λ₂ʰ[r,:,:]))/2
            ELBO[eph] += (M - tr(Λ₂ʰ[r,:,:] \ Λ₂[r,:,:]))/2
            ELBO[eph] += (a₂[r] - a₂ʰ[r])*log_ρ₂[r] - (b₂[r] - b₂ʰ[r])*ρ₂[r]
            ELBO[eph] -= ρ₂[r]*((m₂ʰ[r,:] - m₂[r,:])'*Λ₂[r,:,:]*(m₂ʰ[r,:] - m₂[r,:]))/2
        end
        
        ELBO[eph] += lgamma(sum(γ)) - sum(lgamma,γ)
        ELBO[eph] -= lgamma(sum(γ)+T) - sum(lgamma,γʰ)
        
        p.Σ_R .= 0.0
        p.Σ_X₁ .= 0.0
        p.Σ_X₁² .= 0.0
        p.Σ_X₂² .= 0.0
        p.Σ_X₂ϕ .= 0.0
        p.Σ_ϕϕᵀ .= 0.0
        
        for (t,(x₁, x₂)) ∈ enumerate(zip(X₁, X₂))
            ϕ .= x₁.^(0:M-1)
            for r ∈ 1:R
                log_q[r] = log_θ[r] 
                log_q[r] += (log_ρ₁[r] - ρ₁[r]*(x₁ - m₁ʰ[r])^2 - 1.0 / Λ₁ʰ[r])/2
                log_q[r] += (log_ρ₂[r] - ρ₂[r]*(x₂ - ϕ'*m₂ʰ[r,:])^2 - ϕ'* (Λ₂ʰ[r,:,:] \ ϕ))/2
            end
            
            log_q .-= logsumexp(log_q)
            q .= exp.(log_q)
            
            for r ∈ 1:R                
                p.Σ_R[r] += q[r]
                p.Σ_X₁[r] += q[r]*x₁
                p.Σ_X₁²[r] += q[r]*x₁*x₁
                p.Σ_X₂²[r] += q[r]*x₂*x₂
                p.Σ_X₂ϕ[r,:] .+= q[r]*x₂*ϕ
                p.Σ_ϕϕᵀ[r,:,:] .+= q[r]*ϕ*ϕ'
                
                ELBO[eph] += q[r]*(log_θ[r] - log_q[r])
                ELBO[eph] += q[r]*(log_ρ₁[r] - ρ₁[r]*(x₁ - m₁ʰ[r])^2 - 1.0 / Λ₁ʰ[r])/2
                ELBO[eph] += q[r]*(log_ρ₂[r] - ρ₂[r]*(x₂ - ϕ'*m₂ʰ[r,:])^2 - ϕ'* (Λ₂ʰ[r,:,:] \ ϕ))/2
            end
            _, p.r[t] = findmax(log_q)
        end
                
        ELBO[eph] -= T*log(2π)
        if eph > 1 && ELBO[eph] - ELBO[eph-1] < 1e-6
            ELBO = ELBO[1:eph]
            break
        end
    end
    return ELBO, p.r
end


v_bayes_pseudodata (generic function with 2 methods)

### Code for reversing the prior data

In [ ]:
Xᵖ[:,:,1], Xᵖ[:,:,2] = Xᵖ[:,:,2], Xᵖ[:,:,1] ;

## Checking the new function

In [12]:
mutable struct Particle{ℜ <: Real}
    r::Array{Int,1}
    Σ_R::Array{ℜ,1}
    Σ_X₁::Array{ℜ,1}
    Σ_X₁²::Array{ℜ,1}
    Σ_X₂²::Array{ℜ,1}
    Σ_X₂ϕ::Array{ℜ,2}
    Σ_ϕϕᵀ::Array{ℜ,3}
    log_πₜ::ℜ
    
    function Particle(X::Array{ℜ,2}, R::Int; M::Int=1) where {ℜ <: Real}
        T, N = size(X)
        return new{ℜ}(zeros(Int,T), zeros(ℜ,R), zeros(ℜ,R), zeros(ℜ,R),
                      zeros(ℜ,R), zeros(ℜ,R,M), zeros(ℜ,R,M,M), 0.0)
    end
    
    function Particle(r::Array{Int,1}, Σ_R::Array{Int,1}, Σ_X₁::Array{ℜ,1}, Σ_X₁²::Array{ℜ,1},
                      Σ_X₂²::Array{ℜ,1}, Σ_X₂ϕ::Array{ℜ,2}, Σ_ϕϕᵀ::Array{ℜ,3}, log_πₜ::ℜ) where {ℜ <: Real}
        return new{ℜ}(copy(r), copy(Σ_R), copy(Σ_X₁), copy(Σ_X₁²),
                      copy(Σ_X₂²), copy(Σ_X₂ϕ), copy(Σ_ϕϕᵀ), log_πₜ)
    end
    
    function Particle(p::Particle{ℜ}) where {ℜ <: Real}
        return new{ℜ}(copy(p.r), copy(p.Σ_R), copy(p.Σ_X₁), copy(p.Σ_X₁²),
                      copy(p.Σ_X₂²), copy(p.Σ_X₂ϕ), copy(p.Σ_ϕϕᵀ), p.log_πₜ)
    end 
end

In [13]:
function make_param(Xᵖ::Array{ℜ,3},M::Int) where {ℜ <: Real}
    R, T = size(Xᵖ)[1:end-1]
    #if T != Int((M+1)*M/2)
    #    throw("Prior data size not correct")
    #end
    X = reshape(Xᵖ,:,2)
    p = Particle(X, R; M=M)
    ϕ = zeros(M)
    
    for r ∈ 1:R, t ∈ 1:T
        x₁, x₂ = Xᵖ[r,t,:]
        ϕ .= x₁.^(0:M-1)
        p.Σ_R[r] += 1.0
        p.Σ_X₁[r] += x₁
        p.Σ_X₁²[r] += x₁*x₁
        p.Σ_X₂²[r] += x₂*x₂
        p.Σ_X₂ϕ[r,:] .+= x₂*ϕ
        p.Σ_ϕϕᵀ[r,:,:] .+= ϕ*ϕ'
    end
    
    γ = zeros(R)
    Λ₁, m₁ = zeros(R), zeros(R)
    a₁, b₁ = zeros(R), zeros(R)
    Λ₂, m₂ = zeros(R,M,M), zeros(R,M) 
    a₂, b₂ = zeros(R), zeros(R)
    
    for r ∈ 1:R
        γ[r] = p.Σ_R[r]
        Λ₁[r] = p.Σ_R[r]
        m₁[r] = p.Σ_X₁[r] / Λ₁[r]
        a₁[r] = p.Σ_R[r]/2
        b₁[r] = (p.Σ_X₁²[r] - Λ₁[r]*m₁[r]*m₁[r])/2
        
        Λ₂[r,:,:] .= p.Σ_ϕϕᵀ[r,:,:]
        m₂[r,:] .= Λ₂[r,:,:] \ p.Σ_X₂ϕ[r,:]
        a₂[r] = p.Σ_R[r]/2
        b₂[r] = (p.Σ_X₂²[r] - m₂[r,:]'*Λ₂[r,:,:]*m₂[r,:])/2
    end
    
    param = Dict(:γ=>γ, :m₁=>m₁, :m₂=>m₂, :Λ₁=>Λ₁, 
    :Λ₂=>Λ₂, :a₁=>a₁, :a₂=>a₂, :b₁=>b₁, :b₂=>b₂);
    
    return param
end

make_param (generic function with 1 method)

In [18]:
R, M = 3, 2
Tᵖ = M+1 #Int((M+1)*M/2)
λ₁ᵣ, λ₂ᵣ, γᵣ = 1., 1., 2.
m₁ᵣ, m₂ᵣ = 0., 0.
a₁ᵣ, a₂ᵣ = 1., 1.
b₁ᵣ, b₂ᵣ = 1., 1.

(1.0, 1.0)

In [42]:
Xᵖ = randn(R,Tᵖ,2);

make_param(Xᵖ,M)

Dict{Symbol,Array{Float64,N} where N} with 9 entries:
  :γ  => [3.0, 3.0, 3.0]
  :Λ₁ => [3.0, 3.0, 3.0]
  :Λ₂ => [3.0 0.846391; 3.0 -1.73993; 3.0 -2.43564]…
  :b₁ => [0.291044, 1.00238, 1.09566]
  :a₁ => [1.5, 1.5, 1.5]
  :a₂ => [1.5, 1.5, 1.5]
  :m₂ => [1.30799 -2.97798; -0.369561 -1.93417; -0.379905 -0.72403]
  :b₂ => [0.399773, 0.0106267, 0.47021]
  :m₁ => [0.28213, -0.579976, -0.811881]

In [ ]:
param_original = Dict(:γ=>γᵣ, :m₁=>m₁ᵣ, :m₂=>m₂ᵣ, :λ₁=>λ₁ᵣ, 
    :λ₂=>λ₂ᵣ, :a₁=>a₁ᵣ, :a₂=>a₂ᵣ, :b₁=>b₁ᵣ, :b₂=>b₂ᵣ)

param_new = Dict(:γ=>fill(γᵣ,R), :m₁=>zeros(R), :m₂=>zeros(R,M), :Λ₁=>fill(λ₁ᵣ,R), 
    :Λ₂=>zeros(R,M,M) .+ λ₂ᵣ*reshape(diagm(0 => ones(M)),1,M,M), :a₁=>fill(a₁ᵣ,R), 
    :a₂=>fill(a₂ᵣ,R), :b₁=>fill(b₁ᵣ,R), :b₂=>fill(b₂ᵣ,R));

In [ ]:
#using Base.Random
X = (rand(20, 2) .- 0.5).*5;

In [ ]:
using Random
Random.seed!(100);

In [ ]:
v_bayes(X, R; M=M, param_new...,EPOCHS=100)

In [ ]:
v_bayes(X, R; M=M, make_param(Xᵖ,M)...,EPOCHS=100)

In [ ]:
Random.seed!(100);

In [ ]:
BayesianCausality.v_bayes(X, R; M=M, param_original...,EPOCHS=100)

In [9]:
using Pkg

In [10]:
Pkg.add("GaussianMixtures")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %========>                                ]  18.0 %35.8 % [=================>                       ]  40.1 %57.9 %>      ]  82.7 % Resolving package versions...
 Installed DataAPI ────────── v1.1.0
 Installed Blosc ──────────── v0.5.1
 Installed ScikitLearnBase ── v0.5.0
 Installed JLD ────────────── v0.9.1
 Installed GaussianMixtures ─ v0.3.0
 Installed HDF5 ───────────── v0.12.4
 Installed FileIO ─────────── v1.0.7
 Installed CMakeWrapper ───── v0.2.3
 Installed CMake ──────────── v1.1.2
 Installed Parameters ─────── v0.12.0
 Installed LegacyStrings ──── v0.4.1
 Installed Missings ───────── v0.4.3
  Updating `~/.julia/environments/v1.0/Project.toml`
  [cc18c42c] + GaussianMixtures v0.3.0
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [a74b3585] + Blosc v0.5.1
  [631607c0] + CMake v1.1.2
  [

In [11]:
using GaussianMixtures

┌ Info: Precompiling GaussianMixtures [cc18c42c-b769-54ff-9e2a-b28141a64aae]
└ @ Base loading.jl:1192


In [30]:
X = [randn(100,2); randn(100,2) .+ 4]

200×2 Array{Float64,2}:
  1.31636   -0.168588 
  0.72205    0.570313 
 -0.515718   0.280166 
  0.343045  -0.0624836
 -0.378063   0.903088 
  0.242655   1.09123  
 -2.00935   -0.864192 
  0.186061  -0.0631674
  0.979084   0.530058 
 -0.704092   0.678778 
 -0.256162  -0.209686 
  0.463475  -0.953381 
  0.708883  -0.7628   
  ⋮                   
  4.31538    3.25244  
  3.82664    3.24637  
  3.52063    3.77244  
  4.26628    4.08115  
  2.7297     6.2803   
  4.27932    4.4038   
  3.80515    4.18755  
  3.63458    3.53404  
  4.62455    5.07443  
  5.16587    3.07147  
  2.49503    3.86513  
  5.15258    1.76264  

In [31]:
gmm = GMM(2, 2; kind=:full)

GMM{Float64} with 2 components in 2 dimensions and full covariance
Mix 1: weight 0.500000
 mean: [0.0, 0.0]
 covariance: 2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0
Mix 2: weight 0.500000
 mean: [0.0, 0.0]
 covariance: 2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0


In [32]:
em!(gmm, X; nIter= 10)

┌ Info: Running 10 iterations EM on full cov GMM with 2 Gaussians in 2 dimensions
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtTJ/src/train.jl:242
┌ Info: iteration 1, average log likelihood -5.507782
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtTJ/src/gmms.jl:71
┌ Info: iteration 2, average log likelihood -1.950772
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtTJ/src/gmms.jl:71
┌ Info: iteration 3, average log likelihood -1.950772
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtTJ/src/gmms.jl:71
┌ Info: iteration 4, average log likelihood -1.950772
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtTJ/src/gmms.jl:71
┌ Info: iteration 5, average log likelihood -1.950772
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtTJ/src/gmms.jl:71
┌ Info: iteration 6, average log likelihood -1.950772
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtT

10-element Array{Float64,1}:
 -5.507781653746094 
 -1.9507716333937153
 -1.9507716333937157
 -1.9507716333937157
 -1.9507716333937157
 -1.9507716333937157
 -1.9507716333937157
 -1.9507716333937157
 -1.9507716333937157
 -1.9507716333937157

In [33]:
gmm

GMM{Float64} with 2 components in 2 dimensions and full covariance
Mix 1: weight 0.500000
 mean: [2.01355, 2.09492]
 covariance: 2×2 Array{Float64,2}:
 4.62819  4.00792
 4.00792  5.28411
Mix 2: weight 0.500000
 mean: [2.01355, 2.09492]
 covariance: 2×2 Array{Float64,2}:
 4.62819  4.00792
 4.00792  5.28411


In [25]:
gmmposterior(gmm, X)[1]

100×2 Array{Float64,2}:
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5
 ⋮       
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5
 0.5  0.5

In [21]:
gmm

GMM{Float64} with 5 components in 2 dimensions and full covariance
Mix 1: weight 0.200000
 mean: [0.0684869, 0.0138058]
 covariance: 2×2 Array{Float64,2}:
 1.09085   0.210428
 0.210428  1.11116 
Mix 2: weight 0.200000
 mean: [0.0684869, 0.0138058]
 covariance: 2×2 Array{Float64,2}:
 1.09085   0.210428
 0.210428  1.11116 
⋮
Mix 4: weight 0.200000
 mean: [0.0684869, 0.0138058]
 covariance: 2×2 Array{Float64,2}:
 1.09085   0.210428
 0.210428  1.11116 
Mix 5: weight 0.200000
 mean: [0.0684869, 0.0138058]
 covariance: 2×2 Array{Float64,2}:
 1.09085   0.210428
 0.210428  1.11116 


4 (1:2, 4:5)


In [34]:
Pkg.add("MixtureModels")

 Resolving package versions...


Pkg.Types.ResolverError: Unsatisfiable requirements detected for package MixtureModels [6ab79980]:
 MixtureModels [6ab79980] log:
 ├─possible versions are: [0.1.0-0.1.1, 0.2.0] or uninstalled
 ├─restricted to versions * by an explicit requirement, leaving only versions [0.1.0-0.1.1, 0.2.0]
 └─restricted by julia compatibility requirements to versions: uninstalled — no versions left

In [35]:
]add ScikitLearn

 Resolving package versions...
 Installed NaNMath ──────────────── v0.3.2
 Installed CommonSubexpressions ─── v0.2.0
 Installed Requires ─────────────── v0.5.2
 Installed Mocking ──────────────── v0.7.0
 Installed ElasticPDMats ────────── v0.2.1
 Installed FastGaussQuadrature ──── v0.4.0
 Installed DiffEqDiffTools ──────── v1.3.0
 Installed DiffRules ────────────── v0.0.10
 Installed RecipesBase ──────────── v0.7.0
 Installed ArrayInterface ───────── v1.2.1
 Installed NBInclude ────────────── v2.1.0
 Installed PositiveFactorizations ─ v0.2.2
 Installed DiffResults ──────────── v0.0.4
 Installed LineSearches ─────────── v7.0.1
 Installed NLSolversBase ────────── v7.5.0
 Installed FillArrays ───────────── v0.7.2
 Installed ElasticArrays ────────── v0.4.0
 Installed Optim ────────────────── v0.19.3
 Installed RData ────────────────── v0.6.3
 Installed CodecZlib ────────────── v0.6.0
 Installed DecisionTree ─────────── v0.9.0
 Installed Calculus ─────────────── v0.5.0
 Installed ForwardDif

In [48]:
using GaussianMixtures: GMM
using ScikitLearn: fit!, predict_proba

gmm = fit!(GMM(n_components=3, # number of Gaussians to fit
               kind=:diag), # diagonal covariance matrix (other option: :full)
           X)
argmax(predict_proba(gmm, X),2)

  Iters               objv        objv-change | affected 
-------------------------------------------------------------
      0       4.614381e+02
      1       3.189460e+02      -1.424921e+02 |        2
      2       3.142817e+02      -4.664285e+00 |        2
      3       3.133555e+02      -9.261388e-01 |        2
      4       3.121482e+02      -1.207363e+00 |        2
      5       3.113324e+02      -8.157327e-01 |        2
      6       3.111460e+02      -1.864424e-01 |        2
      7       3.111221e+02      -2.388829e-02 |        0
      8       3.111221e+02       0.000000e+00 |        0
K-means converged with 8 iterations (objv = 311.12211257204854)


┌ Info: Initializing GMM, 3 Gaussians LinearAlgebra.diag covariance 2 dimensions using 200 data points
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtTJ/src/train.jl:77
┌ Info: K-means with 200 data points using 8 iterations
│ 22.2 data points per parameter
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtTJ/src/train.jl:138
┌ Info: Running 10 iterations EM on diag cov GMM with 3 Gaussians in 2 dimensions
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtTJ/src/train.jl:242
┌ Info: iteration 1, average log likelihood -1.725660
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtTJ/src/gmms.jl:71
┌ Info: iteration 2, average log likelihood -1.719060
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtTJ/src/gmms.jl:71
┌ Info: iteration 3, average log likelihood -1.717417
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtTJ/src/gmms.jl:71
┌ Info: iteration 4, average log like

200-element Array{Int64,1}:
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 ⋮
 3
 3
 1
 1
 1
 1
 1
 3
 1
 3
 3
 3

In [60]:
T = 5
Xᵖ = zeros(R,T,2)

3×5×2 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [61]:
X

200×2 Array{Float64,2}:
  1.31636   -0.168588 
  0.72205    0.570313 
 -0.515718   0.280166 
  0.343045  -0.0624836
 -0.378063   0.903088 
  0.242655   1.09123  
 -2.00935   -0.864192 
  0.186061  -0.0631674
  0.979084   0.530058 
 -0.704092   0.678778 
 -0.256162  -0.209686 
  0.463475  -0.953381 
  0.708883  -0.7628   
  ⋮                   
  4.31538    3.25244  
  3.82664    3.24637  
  3.52063    3.77244  
  4.26628    4.08115  
  2.7297     6.2803   
  4.27932    4.4038   
  3.80515    4.18755  
  3.63458    3.53404  
  4.62455    5.07443  
  5.16587    3.07147  
  2.49503    3.86513  
  5.15258    1.76264  

In [62]:
gmm = fit!(GMM(n_components=R, kind=:full), X)
clusters = argmax(predict_proba(gmm, X),2)
unchosen = 1:size(X)[1]
for r ∈ 1:R
    cluster_r = findall(x->x==r, clusters)
    chosen = map(x->x[1],randperm(length(cluster_r))[1:T])
    unchosen = setdiff(unchosen,chosen)
    Xᵖ[r,:,:] .= X[chosen,:] 
end

X = X[unchosen,:]

  Iters               objv        objv-change | affected 
-------------------------------------------------------------
      0       5.738387e+02
      1       3.116034e+02      -2.622353e+02 |        3
      2       3.076599e+02      -3.943442e+00 |        2
      3       3.072643e+02      -3.955874e-01 |        2
      4       3.071640e+02      -1.003578e-01 |        0
      5       3.071640e+02       0.000000e+00 |        0
K-means converged with 5 iterations (objv = 307.1639880723272)


┌ Info: Initializing GMM, 3 Gaussians LinearAlgebra.diag covariance 2 dimensions using 200 data points
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtTJ/src/train.jl:77
┌ Info: K-means with 200 data points using 5 iterations
│ 22.2 data points per parameter
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtTJ/src/train.jl:138
┌ Info: Running 10 iterations EM on full cov GMM with 3 Gaussians in 2 dimensions
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtTJ/src/train.jl:242
┌ Info: iteration 1, average log likelihood -1.725764
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtTJ/src/gmms.jl:71
┌ Info: iteration 2, average log likelihood -1.718216
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtTJ/src/gmms.jl:71
┌ Info: iteration 3, average log likelihood -1.715843
└ @ GaussianMixtures /home/melih/.julia/packages/GaussianMixtures/RGtTJ/src/gmms.jl:71
┌ Info: iteration 4, average log like

185×2 Array{Float64,2}:
  0.72205    0.570313 
 -0.515718   0.280166 
  0.343045  -0.0624836
  0.242655   1.09123  
 -2.00935   -0.864192 
  0.979084   0.530058 
 -0.704092   0.678778 
 -0.256162  -0.209686 
  0.463475  -0.953381 
 -1.52663   -1.08488  
  0.915789  -1.22688  
  0.628634   0.790888 
  1.68144    1.38048  
  ⋮                   
  4.31538    3.25244  
  3.82664    3.24637  
  3.52063    3.77244  
  4.26628    4.08115  
  2.7297     6.2803   
  4.27932    4.4038   
  3.80515    4.18755  
  3.63458    3.53404  
  4.62455    5.07443  
  5.16587    3.07147  
  2.49503    3.86513  
  5.15258    1.76264  

In [55]:
using Random: randperm

In [57]:
chosen

UndefVarError: UndefVarError: chosen not defined